In [ ]:
import os
import joblib
import pandas as pd
import shap
from lime import lime_tabular
import matplotlib.pyplot as plt


In [ ]:
# Load model and data
model = joblib.load('../models/random_forest_fraud_model.pkl')
X_test = pd.read_csv('../data/processed/test_data.csv').drop('is_fraud', axis=1)

In [ ]:
# Create explanations directory
os.makedirs('../explanations/shap', exist_ok=True)
os.makedirs('../explanations/lime', exist_ok=True)

In [ ]:
## SHAP Analysis
# %%
# Initialize SHAP explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
# Summary plot
shap.summary_plot(shap_values[1], X_test, show=False)
plt.savefig('../explanations/shap/summary_plot.png', bbox_inches='tight')
plt.close()

In [ ]:
# Force plot for first prediction
shap.force_plot(
    explainer.expected_value[1], 
    shap_values[1][0], 
    X_test.iloc[0],
    matplotlib=True,
    show=False
)
plt.savefig('../explanations/shap/force_plot.png')  # Alternative: HTML version
# shap.save_html('../explanations/shap/force_plot.html', force_plot)

In [ ]:
# Dependence plot
shap.dependence_plot(
    'transaction_amount', 
    shap_values[1], 
    X_test,
    interaction_index='merchant_category'
)
plt.savefig('../explanations/shap/dependence_plot.png')
plt.close()

In [ ]:
## LIME Analysis
# %%
# Initialize LIME explainer
explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_test.values,
    feature_names=X_test.columns,
    class_names=['Legitimate', 'Fraud'],
    mode='classification'
)

# Explain first instance
exp = explainer.explain_instance(
    X_test.iloc[0].values, 
    model.predict_proba, 
    num_features=10
)

# Save explanation
fig = exp.as_pyplot_figure()
fig.savefig('../explanations/lime/lime_explanation_0.png', bbox_inches='tight')
plt.close()